In [1]:
import torch
from torch import nn
from torch.nn import functional as F

net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
x = torch.rand(2, 20)
net(x)

tensor([[-0.0076,  0.0240, -0.1908,  0.3080, -0.0616, -0.0951,  0.1085,  0.0237,
         -0.0286, -0.0427],
        [ 0.1254,  0.0602,  0.0548,  0.3273, -0.1227, -0.0288,  0.2365, -0.1000,
          0.1985,  0.0518]], grad_fn=<AddmmBackward>)

In [2]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.out = nn.Linear(256, 10)
        
    def forward(self, x):
        return self.out(F.relu(self.hidden(x)))

In [3]:
net = MLP()
net(x)

tensor([[ 0.0160, -0.1805, -0.2310,  0.1939, -0.2472,  0.0334,  0.1293, -0.3164,
          0.0336, -0.3228],
        [-0.0330, -0.2526, -0.0673,  0.1963, -0.1911,  0.0531, -0.0148, -0.3664,
         -0.0533, -0.3180]], grad_fn=<AddmmBackward>)

In [4]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for block in args:
            self._modules[block] = block
            
    def forward(self, x):
        for block in self._modules.values():
            x = block(x)
        return x

In [5]:
net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(x)

tensor([[ 0.0821,  0.0722,  0.1187, -0.0740,  0.2323,  0.0537,  0.0655, -0.0152,
         -0.1487, -0.0887],
        [ 0.0567,  0.1315,  0.2302, -0.1273,  0.3945,  0.0436,  0.0724,  0.1202,
         -0.0137, -0.0216]], grad_fn=<AddmmBackward>)

In [6]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)
        
    def forward(self, x):
        x = self.linear(x)
        x = F.relu(torch.mm(x, self.rand_weight) + 1)
        x = self.linear(x)
        while x.abs().sum() > 1:
            x /= 2
        return x.sum()

In [7]:
net = FixedHiddenMLP()
net(x)

tensor(-0.1392, grad_fn=<SumBackward0>)

In [8]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),
                                nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)
        
    def forward(self, x):
        return self.linear(self.net(x))
    
chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(x)

tensor(-0.2375, grad_fn=<SumBackward0>)